In [1]:
#os and i/o
import os
import numpy as np
import glob
import os.path as op
import pandas as pd
import json

In [2]:
def make_dir(dir_name):
    if not op.exists(dir_name):
        os.mkdir(dir_name)

In [3]:
def make_link(old,new):
    if not op.exists(new):
        cmd = ['ln','-s',old,new]
        cmd = ' '.join(cmd)
        os.system(cmd)

In [31]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
bids_dir = home_dir + '/fd-BIDS/'
make_dir(bids_dir)

In [5]:
#get new sub names
sub_map = {}
for n,sub in enumerate(sub_list):
    
    if n < 9:
        val = '0' + str(n + 1)
    else:
        val = str(n + 1)

    sub_map[sub] = 'sub-' + val

s = pd.DataFrame({'participant_id': sorted(sub_map.values())})
s.to_csv(bids_dir + 'participants.tsv',index = False)

In [6]:
#initialize directories
for sub in sub_list:
    #make new sub dir
    sub_dir = bids_dir + sub_map[sub]
    make_dir(sub_dir)
    
    #make new anat dir
    anat = op.join(sub_dir, 'anat')
    make_dir(anat)
    
    #make new func dir
    func = op.join(sub_dir, 'func')
    make_dir(func) 
   
    #make new fmap dir
    fmap = op.join(sub_dir, 'fmap')
    make_dir(fmap)
    
    #make new fmap dir
    fmap = op.join(sub_dir, 'dwi')
    make_dir(fmap)
    

In [7]:
#set up anat files
pydeface = '/home/iballard/anaconda/bin/pydeface.py'
for sub in sub_list:
    t1 = op.join(home_dir,'data',sub,'anat','T1w_Whole_brain_1mm.nii.gz')
    
    cmd = ' '.join([pydeface, t1])
#     os.system(cmd)


In [8]:
#set up anat files
for sub in sub_list:
    new_anat = op.join(bids_dir, sub_map[sub], 'anat',sub_map[sub] + '_T1w.nii.gz')

    t1 = op.join(home_dir,'data',sub,'anat','T1w_Whole_brain_1mm_defaced.nii.gz')
    make_link(t1,new_anat)


In [49]:
#set up func files
for exp in ['sim','ser','loc']:
    for sub in sub_list:
        func = glob.glob(op.join(home_dir,'data',sub,'func',exp,'*.nii.gz'))

        for f in func:
            if f.count('_') == 2 and f[-8] != 'p':
                run = f.split('/')[-1].split('.')[0][-1]
                new_func = op.join(bids_dir, sub_map[sub], 'func',
                                   sub_map[sub] + '_task-' + exp + '_run-0' + run + '_bold')
#                 make_link(f,new_func + '.nii.gz')
                cmd = ['ln',f,new_func  + '.nii.gz']
                cmd = ' '.join(cmd)
                os.system(cmd)

                #json info
                info = op.join(home_dir,'scan_info.json')
                make_link(info,new_func + '.json')

                #events info
                info = op.join(home_dir,'fd-BIDS','task-' + exp + '_events.json')
                make_link(info,new_func[:-4] + 'events.json')
        

In [44]:
#set up fmap files
for sub in sub_list:
    func = glob.glob(op.join(home_dir,'data',sub,'cal','cal1*.nii.gz'))
    for f in func:
        if f.count('_') == 2:
            run = f.split('/')[-1].split('.')[0][-1]
                        
            if int(run) < 5:
                new_cal = op.join(bids_dir, sub_map[sub], 'fmap',
                                   sub_map[sub] + '_dir-opposing_run-0' + run + '_epi')
                make_link(f,new_cal + '.nii.gz')

                #deal with json file
                if int(run) < 4:
                    json_str  = {"PhaseEncodingDirection": "j-", 
                                 "TotalReadoutTime":  0.05133,
                                 "IntendedFor": ["func/" + sub_map[sub] + "_task-sim_run-0" + run + "_bold.nii.gz","func/" + sub_map[sub] + "_task-ser_run-0" + run + "_bold.nii.gz"]}
                else:
                    json_str  = {"PhaseEncodingDirection": "j-", 
                                 "TotalReadoutTime":  0.05133,
                                 "IntendedFor": ["func/" + sub_map[sub] + "_task-loc_run-01_bold.nii.gz", "func/" + sub_map[sub] + "_task-loc_run-02_bold.nii.gz"]}

                with open(new_cal + '.json', 'w') as outfile:
                    json.dump(json_str, outfile)

In [35]:
#set up dti files
for sub in sub_list:
    func = glob.glob(op.join(home_dir,'data',sub,'dti','*.nii.gz'))
    for f in func:
        run = f.split('/')[-1].split('.')[0][-1]
        new_func = op.join(bids_dir, sub_map[sub], 'dwi',
                           sub_map[sub] + '_run-0' + run + '_dwi')
        make_link(f,new_func + '.nii.gz')
        
        d = f[:-6] + 'bvec'
        new_func = op.join(bids_dir, sub_map[sub], 'dwi',
                           sub_map[sub] + '_run-0' + run + '_dwi')
        make_link(d,new_func + '.bvec')
        

        d = f[:-6] + 'bval'
        new_func = op.join(bids_dir, sub_map[sub], 'dwi',
                           sub_map[sub] + '_run-0' + run + '_dwi')
        make_link(d,new_func + '.bval')
        
        
        #json info
        info = op.join(home_dir,'dwi_info.json')
        make_link(info,new_func + '.json')


        

In [20]:

def get_run_map(run_df, exp, run):

    if exp == 'sim':
        if run == 1:
            run_df['a_stim'] = 'house'
            run_df['b_stim'] = 'face'
            run_df['c_stim'] = 'body'

        elif run == 2:
            run_df['a_stim'] = 'face'
            run_df['b_stim'] = 'body'
            run_df['c_stim'] = 'house'        
        elif run == 3:
            run_df['a_stim'] = 'body'
            run_df['b_stim'] = 'house'
            run_df['c_stim'] = 'face'        
    
    if exp == 'ser':
        if run == 1:
            run_df['a_stim'] = 'face'
            run_df['b_stim'] = 'body'
            run_df['c_stim'] = 'house'

        elif run == 2:
            run_df['a_stim'] = 'body'
            run_df['b_stim'] = 'house'
            run_df['c_stim'] = 'face'        
        elif run == 3:
            run_df['a_stim'] = 'house'
            run_df['b_stim'] = 'face'
            run_df['c_stim'] = 'body'    
    
    return run_df

In [26]:
timing = op.join('/home','iballard','Dropbox','fd','timing_info_PE_diff_one.csv')
timing = pd.read_csv(timing)
timing = timing
timing['onset'] = timing['onset'] + 3
timing['duration'] = 1.5
timing.loc[timing[timing['condition'] == 'feedback'].index,'duration'] = 1.0
timing = timing.fillna('n/a')

for exp in ['sim','ser']:
    exp_df = timing[timing['exp'] == exp]
    for sub in sub_list:
        sub_df = exp_df[exp_df['sub'] == sub]

        for run in set(sub_df['run']):

            run_df = sub_df[sub_df['run'] == run]
            r = get_run_map(run_df, exp, run)

            out_f = op.join(bids_dir, sub_map[sub], 'func',
                            sub_map[sub] + '_task-' + exp + '_run-0' + str(run) + '_events.tsv')

            r = r[[ 'onset','duration', 'condition', 
                    'feedback_valence', 'rew', 'rt', 
                    'PE', 'PE_mb',
                   'a_stim','b_stim','c_stim']]
            r.to_csv(out_f, sep = '\t', index = False)

/home/iballard/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/iballard/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/iballard/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [23]:
##get timing file for object localizer
def load_timing_file(run,time_to_drop):
    loc_file = home_dir + '/loc_timing/script_kidLoc_2Hz_run' + str(run)
    trial_type = []
    time = []
    with open(loc_file) as f:
        for n,line in enumerate(f):
            l = line.strip('\n').split('\t')
            if len(l)==5: #ignore file header and footer
                trial_type.append(int(l[2]))
                time.append(float(l[1]) - time_to_drop)
    return trial_type, time

In [24]:
#trial type dictionary for object localizer
def trial_type_dict(cond):
    tt_dict = {0:'scramble', 1:'face-adult', 2:'face-child', 3:'body', 4:'limb', 5:'car',
            6:'guitar', 7:'place', 8:'house', 9:'word', 10:'number'}
    c = [tt_dict[x] for x in cond]
    return c

In [25]:
for sub in sub_list:
    design_dir = home_dir + 'data/' + sub + '/design/' #make design folder

    time_to_drop = 0

    #prepare out file
    out_file = design_dir + 'loc.csv'
    
    for run in range(1,3):
        df = {'condition':[],'onset':[]}
        trial_type, time = load_timing_file(run,time_to_drop)
        trial_type = trial_type_dict(trial_type)
        trial_type = trial_type[0::8]
        time = time[0::8]
        
        df['condition'].extend(trial_type)
        df['onset'].extend(time)
        
        df = pd.DataFrame(df)
        df['onset'] = df['onset'] + 15
        df['duration'] = 4
        
        df = df[['onset','duration','condition']]
        out_f = op.join(bids_dir, sub_map[sub], 'func',
                                sub_map[sub] + '_task-loc_run-0' + str(run) + '_events.tsv')
        df.to_csv(out_f, sep = '\t', index = False)